# Dataset & DataLoader

## 1. 概念和用途
用于处理数据样本的代码可能会变得混乱且难以维护;理想情况下，我们需要我们的数据集代码与我们的模型训练代码分离，以获得更好的可读性和模块化。

PyTorch 提供了两个数据抽象类：`torch.utils.data.DataLoader` 和 `torch.utils.data.Dataset` ，允许使用预加载的数据集或者自己的数据。`Dataset` 用于存储样本及其相应的标签，而 `DataLoader` 则在 `Dataset` 外部封装了一个可迭代器，以方便访问样本。 [Dataset & DataLoader 官方解释](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html)

官方解释可能比较抽象，可以结合 Python 的字典来理解。在索引字典中的元素时，其实是调用了 `__getitem__` 方法，如下所示：

In [6]:
dic = {
    "name": "Siyu",
    "age": "23"
}
print(dic["name"])
print(dic.__getitem__("age"))

Siyu
23


那么在存储数据集时，可以采用类似字典的存储方式，然而神经网络中的数据集，在索引往往需要返回两个值：数据（data）和对应的标签（label/target），所以需要重写 `__getitem__` 方法，`Dataset` 抽象类就进行了重写。

所以 `Dataset` 就是专用于实例化存放数据集对象的抽象类。

在训练时，只有 `Dataset` 数据集并不足够，因为需要进行循环，`Dataset` 并未实现迭代器，所以就需要使用 `DataLoader` 帮助数据集能够迭代。

## 2. Dataset
PyTorch 为我们提供了一些已经封装好的数据集，不需要我们自己定义。CIFAR10 是 CV 训练中经常使用到的一个数据集，在 PyTorch 中 CIFAR10 是一个写好的 `Dataset`，我们使用时只需以下代码：

In [ ]:
data = torchvision.datasets.CIFAR10("./data/", transform=torchvision.transforms.ToTensor(), train=True, download=True)

`datasets.CIFAR10` 就是一个 `Dataset` 子类，`data` 是这个类的一个实例。

如果要使用自己的数据集，那么就可以自己定义数据集类。所谓数据集，其实就是一个负责处理索引(index)到样本(sample)映射的一个类(class)。

`torch.utils.data.Dataset` 是一个表示数据集的抽象类。任何自定义的数据集都需要继承这个类并覆写相关方法。

Pytorch提供两种数据集： Map式数据集 Iterable式数据集

### Map Dataset
一个Map式的数据集必须要重写 `getitem(self, index)` , `len(self)` 两个内建方法，用来表示从索引到样本的映射（Map）.

这样一个数据集 dataset，举个例子，当使用 `dataset[idx]` 命令时，可以在你的硬盘中读取你的数据集中第 idx 张图片以及其标签（如果有的话）; `len(dataset)` 则会返回这个数据集的容量。

自定义类大致是这样的：


In [ ]:
class CustomDataset(data.Dataset):#需要继承data.Dataset
    def __init__(self):
        # TODO
        # 1. Initialize file path or list of file names.
        pass
    def __getitem__(self, index):
        # TODO
        # 1. Read one data from file (e.g. using numpy.fromfile, PIL.Image.open).
        # 2. Preprocess the data (e.g. torchvision.Transform).
        # 3. Return a data pair (e.g. image and label).
        #这里需要注意的是，第一步：read one data，是一个data
        pass
    def __len__(self):
        # You should change 0 to the total size of your dataset.
        return 0

Tudui 教学中的例子：

In [ ]:
class MyData(Dataset):
    def __init__(self, img_dir: str, label: str):
        self.__img_dir = img_dir
        self.img_name_list = os.listdir(img_dir)
        self.label = label

    def __getitem__(self, index):
        img_name = self.img_name_list[index]
        img_path = os.path.join(self.__img_dir, img_name)
        img = Image.open(img_path)
        label = self.label
        return img, label

    def __len__(self):
        return len(self.img_name_list)

### Iterable Dataset
暂时用不到。

### 常见的 Transforms
torchvision.transforms : 常用的图像预处理方法，提高泛化能力，功能包括：，数据中心化，数据标准化，缩放，裁剪，旋转，翻转，填充，噪声添加，灰度变换，线性变换，仿射变换，亮度、饱和度及对比度变换。
采用transforms.Compose()，将一系列的transforms有序组合，实现时按照这些方法依次对图像操作。

In [ ]:
train_transform = transforms.Compose([
    transforms.Resize((32, 32)),  # 缩放
    transforms.RandomCrop(32, padding=4),  # 随机裁剪
    transforms.ToTensor(),  # 图片转张量，同时归一化0-255 ---》 0-1
    transforms.Normalize(norm_mean, norm_std),  # 标准化均值为0标准差为1
])

官方文档：[Transforming and augmenting images](https://pytorch.org/vision/stable/transforms.html)

## 3. Dataloader
> Data loader. Combines a dataset and a sampler, and provides an iterable over the given dataset. --PyTorch Documents

一般来说PyTorch中深度学习训练的流程是这样的： 1. 创建Dateset 2. Dataset传递给DataLoader 3. DataLoader迭代产生训练数据提供给模型。

对应的一般都会有这三部分代码

In [ ]:
# 创建Dateset(可以自定义)
    dataset = face_dataset # Dataset部分自定义过的face_dataset
# Dataset传递给DataLoader
    dataloader = torch.utils.data.DataLoader(dataset,batch_size=64,shuffle=False,num_workers=8)
# DataLoader迭代产生训练数据提供给模型
    for i in range(epoch):
        for index,(img,label) in dataloader:
            pass

到这里应该就 PyTorch 的数据集和数据传递机制应该就比较清晰明了了。Dataset 负责建立索引到样本的映射，DataLoader 负责以特定的方式从数据集中迭代的产生一个个 batch 的样本集合。在循环过程中实际上是 dataloader 按照其参数规定的策略调用了其 dataset 的 getitem 方法。

### 参数
先看一下实例化一个DataLoader所需的参数，我们只关注几个重点即可。

In [ ]:
DataLoader(dataset, batch_size=1, shuffle=False, sampler=None,
           batch_sampler=None, num_workers=0, collate_fn=None,
           pin_memory=False, drop_last=False, timeout=0,
           worker_init_fn=None)

In [ ]:
DataLoader(dataset, batch_size=1, shuffle=False, sampler=None,
           batch_sampler=None, num_workers=0, collate_fn=None,
           pin_memory=False, drop_last=False, timeout=0,
           worker_init_fn=None)

参数介绍： 
- dataset (Dataset) – 定义好的Map式或者Iterable式数据集。 
- batch_size (python:int, optional) – 一个batch含有多少样本 (default: 1)。 
- shuffle (bool, optional) – 每一个epoch的batch样本是相同还是随机 (default: False)。 如果设置shuffle=True，那么在下一次epoch时，会将数据打乱顺序，然后再进行下一次读取，从而两次epoch读到的数据顺序是不同的；如果设置shuffle=False，那么在下一次数据读取时，不会打乱数据的顺序，从而两次读取到的数据顺序是相同的。
- sampler (Sampler, optional) – 决定数据集中采样的方法. 如果有，则shuffle参数必须为False。 
- batch_sampler (Sampler, optional) – 和 sampler 类似，但是一次返回的是一个batch内所有样本的index。和 batch_size, shuffle, sampler, and drop_last 三个参数互斥。 
- num_workers (python:int, optional) – 多少个子程序同时工作来获取数据，多线程。 (default: 0) 
- collate_fn (callable, optional) – 合并样本列表以形成小批量。 在dataloader构建的时侯，collate_fn一般是不用特殊指明的，因为默认的方法会将数据组织成我们想要的方式。但是如果想定制**batch的输出形式**的话，这个参数就非常重要了。比如在3D目标检测中，我们的batch形式希望是pointcloud_batch，gt_boxes_batch这种形式的时侯。
- pin_memory (bool, optional) – 如果为True，数据加载器在返回前将张量复制到CUDA固定内存中。 
- drop_last (bool, optional) – 如果数据集大小不能被batch_size整除，设置为True可删除最后一个不完整的批处理。如果设为False并且数据集的大小不能被batch_size整除，则最后一个batch将更小。(default: False) 
- timeout (numeric, optional) – 如果是正数，表明等待从worker进程中收集一个batch等待的时间，若超出设定的时间还没有收集到，那就不收集这个内容了。这个numeric应总是大于等于0。 (default: 0) 
- worker_init_fn (callable, optional*) – 每个worker初始化函数 (default: None)